# Azure OpenAI Pricing Tool

Fetch Azure OpenAI pricing data using the Azure Retail Prices API.

## Features
- 🔍 Search for specific models (e.g., GPT-4o, GPT-4 Turbo)
- 🌍 Filter by Azure regions
- 💱 Support for multiple currencies
- 📊 Clean, formatted output
- 🔄 Automatic pagination handling

### Parameters for `get_aoai_prices()`:
- `region` - Azure region (e.g., "swedencentral", "westeurope")  
- `model_substr` - Model name substring (e.g., "gpt-4o", "GPT-4 Turbo")
- `currency` - Currency code (e.g., "USD", "EUR", "SEK")
- `product_name` - Service name (default: "Azure OpenAI")

In [1]:
from datetime import datetime
from typing import Dict, List

import requests
import pandas as pd

_API_ROOT = "https://prices.azure.com/api/retail/prices"
_API_VERSION = "2023-01-01-preview"
_HEADERS = {"User-Agent": "aoai-pricing-jupyter/1.0"}


def _get_all_items(params: Dict) -> List[Dict]:
    """Get all price rows, handling pagination automatically."""
    all_items = []
    url = _API_ROOT  # Initialize the URL!

    while url:
        response = requests.get(
            url,  # Use the current URL (first time: _API_ROOT, later: NextPageLink)
            params=params if url == _API_ROOT else None,  # Only send params on first request
            headers=_HEADERS,
            timeout=30,
        )
        response.raise_for_status()
        data = response.json()

        all_items.extend(data["Items"])
        url = data.get("NextPageLink")  # None when no more pages, required if result set>1000 items

    return all_items


def get_aoai_prices(
    region: str = "swedencentral",
    model_substr: str = "gpt-4o",
    currency: str = "USD",
    product_name: str = "Azure OpenAI",
) -> pd.DataFrame:
    """
    Return a DataFrame of Azure OpenAI retail prices.

    Parameters
    ----------
    region : str
        Azure ARM region code (e.g. 'westeurope', 'swedencentral').
    model_substr : str
        Substring matched against `meterName`. Case-sensitive!!!!
    currency : str
        ISO currency code ('USD', 'EUR', 'SEK', …).
    product_name : Azure OpenAI
        The product name to filter by, e.g. 'Azure OpenAI'.
    """
    filters = [
        f"armRegionName eq '{region}'",
        f"productName eq '{product_name}'",
        f"contains(meterName,'{model_substr}')",
        "priceType eq 'Consumption'",
    ]
    params: Dict[str, str] = {
        "api-version": _API_VERSION,
        "$filter": " and ".join(filters),
        "currencyCode": currency,
    }

    rows = _get_all_items(params)
    if not rows:
        raise ValueError(
            "No meters matched your filters. "
            "Double-check the region spelling and be sure the model substring's case matches "
            "exactly what the portal shows (e.g. 'GPT-4o', not 'gpt-4o')."
        )

    df = pd.DataFrame(rows)
    df["price_per_million"] = df["unitPrice"] * 1_000
    return df[
        [
            "meterName",
            "armRegionName",
            "currencyCode",
            "unitOfMeasure",
            "unitPrice",
            "price_per_million",
        ]
    ].sort_values("meterName")


print("Helper loaded:", datetime.utcnow().strftime("%Y-%m-%d %H:%M UTC"))

Helper loaded: 2025-07-07 10:16 UTC


C:\Users\povelf\AppData\Local\Temp\ipykernel_10620\1571887516.py:87: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  print("Helper loaded:", datetime.utcnow().strftime("%Y-%m-%d %H:%M UTC"))


In [2]:
regions = ["swedencentral", "westeurope"]
dfs = []

for region in regions:
    df = get_aoai_prices(region=region, model_substr="gpt-4o", currency="USD")
    dfs.append(df)

## Display Results

In [3]:
for region in regions:
    print(f"{"==="*15} {region.upper()} {"==="*15}")
    print(f"{'Model':<52} {'Unit':<12} {'$/1K tokens':<12} {'$/1M tokens':<12} {'Region':<15}")
    print("-" * 105)

    for _, row in df.iterrows():
        model_name = row['meterName'][:50] + "..." if len(row['meterName']) > 33 else row['meterName']
        print(
            f"{model_name:<52} "
            f"{row['unitOfMeasure']:<12} "
            f"${row['unitPrice']:<11.6f} "
            f"${row['price_per_million']:<11.2f} "
            f"{row['armRegionName']:<15}"
        )
    print("\n")

============================================= SWEDENCENTRAL =============================================
Model                                                Unit         $/1K tokens  $/1M tokens  Region         
---------------------------------------------------------------------------------------------------------
gpt-4o-0806-Batch-Inp-glbl Tokens                    1K           $0.001250    $1.25        westeurope     
gpt-4o-0806-Batch-Outp-glbl Tokens...                1K           $0.005000    $5.00        westeurope     
gpt-4o-0806-Inp-glbl Tokens                          1K           $0.002500    $2.50        westeurope     
gpt-4o-0806-Outp-glbl Tokens                         1K           $0.010000    $10.00       westeurope     
gpt-4o-64k-outp-alpha-0730-Inp-glbl Tokens...        1K           $0.006000    $6.00        westeurope     
gpt-4o-64k-outp-alpha-0730-Outp-glbl Tokens...       1K           $0.018000    $18.00       westeurope     
gpt-4o-aud-0603 Inp DZone Tokens